<a href="https://colab.research.google.com/github/alabhastam/semantic-text-indexing/blob/main/semantic_text_indexing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q faiss-cpu sentence-transformers transformers pymupdf gradio



In [3]:
import fitz
from google.colab import files

def extract_text_from_pdf(file_path):
    doc = fitz.open(file_path)
    return "\n".join([page.get_text() for page in doc])

uploaded_files = files.upload()
all_text = ""
for file_name in uploaded_files.keys():
    all_text += extract_text_from_pdf(file_name) + "\n\n"


Saving @MachineLearning_ir - Mathematics of ML.pdf to @MachineLearning_ir - Mathematics of ML.pdf
Saving @MachineLearning_ir_Essentials_of_Probability_&_Statistics.pdf to @MachineLearning_ir_Essentials_of_Probability_&_Statistics.pdf
Saving @MachineLearning_ir_Mathematics_for_ML_Essential_Equations.pdf to @MachineLearning_ir_Mathematics_for_ML_Essential_Equations.pdf


In [ ]:
from langchain.text_splitter import CharacterTextSplitter
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_text(all_text)

embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
doc_embeddings = embedding_model.encode(chunks).astype("float32")

index = faiss.IndexFlatL2(doc_embeddings.shape[1])
index.add(doc_embeddings)


In [ ]:
from transformers import pipeline
qa_pipeline = pipeline("question-answering", model="distilbert-base-cased-distilled-squad")

def ask_question(query):
    q_emb = embedding_model.encode([query]).astype("float32")
    _, I = index.search(q_emb, k=3)
    context = "\n".join([chunks[i] for i in I[0]])

    result = qa_pipeline({"context": context, "question": query})
    return result["answer"]


In [8]:
import gradio as gr

demo = gr.Interface(fn=ask_question,
                    inputs=gr.Textbox(lines=2, placeholder="Ask a question..."),
                    outputs="text",
                    title=" Multi-PDF QA System")

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://27f5b7d37a0b9af504.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
